In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

### prepare the data set


In [2]:
files=[file for file in os.listdir('./Sales_Data/')]
df=pd.DataFrame()
for i in files:
    d=pd.read_csv('./Sales_Data/'+i)
    df=pd.concat([df,d])
df=df.dropna()
df=df[df['Order Date'].str[0:2]!='Or']

df['month']=df['Order Date'].str[0:2].astype('int32')

df['Quantity Ordered']=pd.to_numeric(df['Quantity Ordered'])
df['Price Each']=pd.to_numeric(df['Price Each'])

df['sales']=df['Quantity Ordered']*df['Price Each']
df['city']=df['Purchase Address'].apply(lambda x: x.split(',')[1])
df['state']=df['Purchase Address'].apply(lambda x: x.split(',')[2])
df['state']=df['state'].str[0:3]
df['city and state']=df['state']+','+df['city']
df.drop(columns=['city','state'],inplace=True)
df['Order Date']=pd.to_datetime(df['Order Date'])
df.head(5)


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,month,sales,city and state
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,23.90,"TX, Dallas"
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,99.99,"MA, Boston"
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,600.00,"CA, Los Angeles"
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,11.99,"CA, Los Angeles"
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,11.99,"CA, Los Angeles"


### what is the best month of sales and how much


In [110]:
best_mont_of_sales=df.groupby('month').sum()
#best_mont_of_sales.to_csv('what is the best month of sales and how much.csv')
best_mont_of_sales['sales'].sum()
#december with 4613443.34 $

34492035.97

### What city sold the most product?

In [89]:
city_solds=df.groupby('city and state').sum()
city_solds.to_csv('What city sold the most product.csv')
city_solds
#San Francisco with 50239 Quantity ordered

,Quantity Ordered,Price Each,month,sales,minute,day,hour
city and state,,,,,,,
"CA, Los Angeles",33289,5421435.23,208325,5452570.80,866638,469607,427444
"CA, San Francisco",50239,8211461.74,315520,8262203.91,1319477,702669,643265
"GA, Atlanta",16602,2779908.20,104794,2795498.58,442932,234837,214264
"MA, Boston",22528,3637409.77,141112,3661642.01,590442,312376,288225
"ME, Portland",2750,447189.25,17144,449758.27,72856,38288,35211
"NY, New York City",27932,4635370.83,175741,4664317.43,733598,392706,357696
"OR, Portland",11303,1860558.22,70621,1870732.34,295533,159233,144421
"TX, Austin",11153,1809873.61,69829,1819581.75,289060,156782,141946
"TX, Dallas",16730,2752627.82,104620,2767975.40,435155,234435,214390


### What time should we display advertisemens to maximize the likelihood of customer’s buying product?

In [109]:
df['hour']=df['Order Date'].dt.hour
results=df.groupby('hour').count()
results
results.to_csv('What time should we display advertisemens.csv')

### what products are most ofen sold together

In [66]:
result=df[df['Order ID'].duplicated(keep=False)]
result['grouped products']=result.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
result=result[['Order ID','grouped products']].drop_duplicates()
result.head(5)


,Order ID,grouped products
3,176560,"Google Phone,Wired Headphones"
18,176574,"Google Phone,USB-C Charging Cable"
30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea..."
32,176586,"AAA Batteries (4-pack),Google Phone"
119,176672,"Lightning Charging Cable,USB-C Charging Cable"


In [105]:
from itertools import combinations
from collections import Counter
count=Counter()
for row in result['grouped products']:
    row_list=row.split(',')
    count.update(Counter(combinations(row_list,2)))
count
produtcts_sold_together=pd.DataFrame()
produtcts_sold_together=produtcts_sold_together.append(count.most_common(10),ignore_index=True)
produtcts_sold_together

produtcts_sold_together.to_csv('what products are most ofen sold togethe2.csv')


#for key, value in count.most_common(10):
 #   print(key, value)



### what product sold the most and why do you think it sold the most?


In [102]:
products_sold_the_mosts=df.groupby('Product').sum()
#products_sold_the_mosts.to_csv('what product sold the most and why do you think it sold the most.csv')
products_sold_the_mosts
#AAA Batteries (4-pack) 	 because the whole world catching up with technology 31017

,Quantity Ordered,Price Each,month,sales,hour
Product,,,,,
20in Monitor,4129,451068.99,29336,454148.71,58764
27in 4K Gaming Monitor,6244,2429637.70,44440,2435097.56,90916
27in FHD Monitor,7550,1125974.93,52558,1132424.50,107540
34in Ultrawide Monitor,6199,2348718.19,43304,2355558.01,89076
AA Batteries (4-pack),27635,79015.68,145558,106118.40,298342
AAA Batteries (4-pack),31017,61716.59,146370,92740.83,297332
Apple Airpods Headphones,15661,2332350.00,109477,2349150.00,223304
Bose SoundSport Headphones,13457,1332366.75,94113,1345565.43,192445
Flatscreen TV,4819,1440000.00,34224,1445700.00,68815
